# EC520 Image Processing and Communication
## Privacy Filter
Cameron Cipriano, Molly Housego

Depth-Driven Computational Imaging: Privacy Filter

The goal of the privacy filter is to identify people's faces in a scene and minimally distort them such that a facial recognition system will be unable to determine who it is. To perform minimal facial distortion, RGB and RGB+D Images were captured of a scene:
1. Facial Detection finds each face present in the image
2. Facial bounding boxes projected into RGB+D images to determine which points are facial pixels
3. Facial pixel coordinates reprojected to 2D image to define outline of face
4. Blur is applied to facial region and non-facial pixels are re-inserted to only distort the face.

Facial Detection implemented using a Haar Cascade Classifier

Facial recognition was implemented using a covariance matrix approach with Nearest Neighbor matching.

## Imports

In [76]:
import os
import cv2 as cv
import mediapipe as mp
import numpy as np
from glob import glob
import pptk
import csv
from tqdm.notebook import tqdm

## Helper Functions

In [118]:
def cleanDirectories(bbox_path, blur_path):
    bbox_image_names = glob(os.path.join(bbox_path, '*.jpg'))
    blur_image_names = glob(os.path.join(blur_path, '*.jpg'))

    for (bbox_img, blur_img) in zip(bbox_image_names, blur_image_names):
        try:
            os.remove(bbox_img)
            os.remove(blur_img)
        except OSError as e:
            print(f'Error: {e.strerror}')

def loadPointClouds(path):
    pointcloud_filenames = sorted(glob(os.path.join(path, '*.csv')))
    pointclouds = {}
    # Setup Progress bar
    iterations = tqdm(pointcloud_filenames, unit='Files')
    for file_idx, csvfile in enumerate(iterations):
        # each point cloud is 39,963 points, each with xyz and rgb values
        pointclouds[file_idx] = {'points': np.zeros((39963,3)), 'colors': np.zeros((39963,3))}
        with open(csvfile, newline='') as pointcloud_file:
            iterations.set_description(f"Parsing: '{pointcloud_file.name}'")
            point_reader = csv.reader(pointcloud_file)
            for point_idx, point_vals in enumerate(point_reader):
                # xyx for current point cloud
                pointclouds[file_idx]['points'][point_idx, 0] = point_vals[0]
                pointclouds[file_idx]['points'][point_idx, 1] = point_vals[1]
                pointclouds[file_idx]['points'][point_idx, 2] = point_vals[2]

                # RGB for current point cloud
                pointclouds[file_idx]['colors'][point_idx, 0] = point_vals[3]
                pointclouds[file_idx]['colors'][point_idx, 1] = point_vals[4]
                pointclouds[file_idx]['colors'][point_idx, 2] = point_vals[5]
    
    return pointclouds

def loadImages(path):
    # Read Images containing faces
    image_filenames = sorted(glob(os.path.join(path, '*.jpg')))
    
    # Setup Progress bar
    image_iterations = tqdm(image_filenames, unit='Images')
    
    images = []
    for filename in image_iterations:
        image_iterations.set_description(f"Loading: '{filename}'")
        images.append(cv.imread(filename))
    
    return np.array(images)
    

## Main Algorithm Functions

In [210]:
def detectFaces(images, classifier):
    """
    Function for detecting faces

    Returns list of rectangles
    """
    detection_iterations = tqdm(images, unit='Image')

    face_bboxes = {}
    detected_faces = []
    for img_idx, frame in enumerate(detection_iterations):
        detection_iterations.set_description('Detecting Faces')
        frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

        # Histogram equalization to improve contrast and make grayscale image more uniform
        frame_gray = cv.equalizeHist(frame_gray)

        #-- Detect faces
        faces = classifier.detectMultiScale(frame_gray, scaleFactor=1.01, minNeighbors=7, minSize=(175, 175), maxSize=(300, 300), flags=cv.CASCADE_SCALE_IMAGE)
        # faces = classifier.detectMultiScale(frame_gray, scaleFactor=1.05, minNeighbors=6, minSize=(175, 175), maxSize=(300, 300), flags=cv.CASCADE_SCALE_IMAGE)

        for (x,y,w,h) in faces:
            # COLOR IS BGR!
            frame = cv.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)

        face_bboxes[img_idx] = faces
        detected_faces.append(frame)

    return np.array(detected_faces), face_bboxes

def recoverFaceOutlines(images, bboxes, P_mat, pointclouds):
    pointcloud_viewer = pptk.viewer(pointclouds[2]['points'], pointclouds[2]['colors']/255)
    pointcloud_viewer.set(lookat=np.zeros((3,1)), phi=-np.pi/2, theta=0, point_size=2)
    pointcloud_viewer.wait()
    pointcloud_viewer.close()

def blurFaceOutlines():
    # blurLevels = np.ones((1, images.shape[1]))
    # images_with_blurred_faces = np.empty(())
    # for img_idx, blurLevel in enumerate(blurLevels):
    #     images_with_blurred_faces[img_idx] = blurFaces(images_with_faces, face_indices, blurLevel)
    pass



def blurRectangularRegion(frame, regions):
    for region in regions:
        (x, y, w, h) = region
        ROI = frame[y:y+h, x:x+w]
        blur = cv.GaussianBlur(ROI, (51, 51), 5)
        frame[y:y+h, x:x+w] = blur

    return frame

## Runner

In [212]:
input_img_path = 'images/jpg'
blurred_img_path = 'images/jpg/blurred'
bbox_img_path = 'images/jpg/bounding_boxes'
pointcloud_path = 'pointclouds'

# Make sure output directories are empty
cleanDirectories(bbox_img_path, blurred_img_path)

# Load in the Facial Detection classifiers
face_cascade_alt2 = cv.CascadeClassifier('cascades/haarcascade_frontalface_alt2.xml')

images                         = loadImages(input_img_path)
pointclouds                    = loadPointClouds(pointcloud_path)
images_with_faces, face_bboxes = detectFaces(images, face_cascade_alt2)
output_tbd                     = recoverFaceOutlines(images, face_bboxes, None, pointclouds)
# blurred_images                 = blurFaceOutlines(images, face_bboxes, output_tbd)


for i in range(len(images_with_faces)):
    width = int(images_with_faces[i].shape[1]* 0.4)
    height = int(images_with_faces[i].shape[0] * 0.4)

    img_tree = cv.resize(images_with_faces[i], (width, height), interpolation=cv.INTER_AREA)
    
    cv.imshow('Tree', img_tree)
    cv.waitKey(0)

cv.destroyAllWindows()
cv.waitKey(1)

  0%|          | 0/8 [00:00<?, ?Images/s]

-1